# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning I</font>

## Usando CNNs Para Reconhecimento da Posição dos Olhos em Fotos

In [ ]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Parte 1) Localizando a posição dos olhos

Parte 2) Segmentação da face em regiões de interesse 

Dataset: https://www.bioid.com/About/BioID-Face-Database

In [ ]:
# Verificando se a GPU tem memória disponível
#!nvidia-smi

In [ ]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [ ]:
# Imports
import os
import re
import numpy as np
import pandas as pd
from itertools import islice
from random import randint
from PIL import Image, ImageOps
import tensorflow
import keras
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Reshape
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

In [ ]:
# Definindo as variáveis que receberão as coordenadas dos olhos e criando um array preenchido com zeros

lx=[]
ly=[]
rx=[]
ry=[]

# Reshape das imagens em 180 x 180
images_orig = np.ndarray(shape = (1521,180,180,1), dtype = np.float32)
print(images_orig)

i=0

### Carregando o Dataset

https://www.bioid.com/About/BioID-Face-Database

In [ ]:
for root, dirnames, filenames in os.walk("/media/datasets/DeepLearningI/Cap09/BioID-FaceDatabase"):
    for filename in filenames:
        if re.search("\.(pgm)$", filename):  # Buscando pelos arquivos de imagem
            
            # Coleta caminho dos arquivos
            filepath = os.path.join(root, filename)
            
            # Localizando as imagens
            image = (load_img(str(filepath.split(".")[0]+".pgm"))) 

            # Crop das imagens para escala left, top, right, bottom
            image = image.crop((102,0,282,180))  
            
            # Reconverte as imagens para esclaa de cinza
            image=ImageOps.grayscale(image)  
            
            # Converte imagens em arrays
            image = img_to_array(image)  
            
            # Append imagem ao numpy array
            images_orig[i] = image   
            print(images_orig)

            # Coleta os arquivos .eye que contém as coordeandas da posição dos olhos
            f = open(str(filepath.split(".")[0]+".eye"),mode="r")  
            for line in islice(f,1,2):
                label = list(map(int, line.split()))  
            lx.append(label[0]-102)
            ly.append(label[1])
            rx.append(label[2]-102)
            ry.append(label[3])
            f.close()
            i += 1
            if i % 250 == 0:
                print("%d imagens no array" % i)
    print("Todas as imagens no array!")

In [ ]:
# Print do resultado da carga de dados

for i in range (0,10):
    fig,ax = plt.subplots(1)

    # Display the image
    index=randint(0,1500)
    ax.imshow(images_orig[index].reshape(180,180),cmap='gray') 

    # Create a Rectangle patch
    rect1 = patches.Rectangle((rx[index],ry[index]),2,2,linewidth=1,edgecolor='r',facecolor='none')
    rect2 = patches.Rectangle((lx[index],ly[index]),2,2,linewidth=1,edgecolor='r',facecolor='none')

    # Add the patch to the Axes
    ax.add_patch(rect1)
    ax.add_patch(rect2)

    plt.show()

Em seguida, normalizamos nossos dados e preparamos para alimentar nosso modelo

In [ ]:
images_orig = images_orig/255.0

Exigimos que nossas variáveis de saída estejam entre 0 e 1 para serem alimentadas em nosso modelo. Isso é para garantir que nossos pesos do modelo não explodem durante o backpropagation.

In [ ]:
# Normalizando coordenadas individuais usando min-max scaler
rx=np.asarray(rx)
rx_max=rx.max()
rx_min=rx.min()
rx=(rx-rx_min)/(rx_max-rx_min)

ry=np.asarray(ry)
ry_max=ry.max()
ry_min=ry.min()
ry=(ry-ry_min)/(ry_max-ry_min)

lx=np.asarray(lx)
lx_max=lx.max()
lx_min=lx.min()
lx=(lx-lx_min)/(lx_max-lx_min)

ly=np.asarray(ly)
ly_max=ly.max()
ly_min=ly.min()
ly=(ly-ly_min)/(ly_max-ly_min)

Em seguida, dividimos nossos dados em conjunto de treino e teste, usamos as primeiras 900 imagens para treinar nosso modelo e usaremos o resto das imagens para testar nosso modelo.

In [ ]:
X_train=images_orig[0:900]
X_test=images_orig[900:1200]

# Reshape dos dados para que estejam compatíveis com o TensorFlow
X_train=X_train.reshape(900,1,180,180)
X_test=X_test.reshape(300,1,180,180)

rx_train=rx[0:900]
rx_test=rx[900:1200]

ry_train=ry[0:900]
ry_test=ry[900:1200]

lx_train=lx[0:900]
lx_test=lx[900:1200]

ly_train=ly[0:900]
ly_test=ly[900:1200]

Em seguida, definimos nosso modelo.

Usaremos os seguintes parâmetros para treinar nosso modelo.

* batch size = 32 - os pesos serão atualizados após 32 amostras
* number epochs = 50 - vamos treinar nosso algoritmo para 50 iterações 
* kernel size = 3 - nós usaremos kernels (filtros) 3 x 3 
* pool size = 2 - usaremos pooling 2x2 
* convolutional layer 1 depth = 32  
* convolutional layer 2 depth = 64  
* convolutional layer 3 depth = 128  
* dropout 1= 0.25 - dropout depois da camada de pooling com probabilidade 0.25  
* dropout 2 = 0.5 - dropout na camada FC layer com probabilidade 0.5  
* hidden layer 1 = 256  - a primeira camada totalmente conectada terá 256 neurônios
* hidden layer 2 = 512  - a segunda camada totalmente conectada terá 512 neurônios

In [ ]:
batch_size = 32 
num_epochs = 50 
kernel_size = 3
pool_size = 2 
conv_depth_1 = 32 
conv_depth_2 = 64 
conv_depth_3 = 128
drop_prob_1 = 0.25 
drop_prob_2 = 0.5 
hidden_size1 = 256 
hidden_size2 = 512

In [ ]:
# Input shape para a camada de entrada no TensorFlow
inp = Input(shape = (1,180,180))

Arquitetura da Primeira Camada Convolucional

Conv_32 -> Conv_32 -> Pool_2x2 -> dropout_0.25

In [ ]:
conv_1 = Convolution2D(conv_depth_1,
                       (kernel_size, kernel_size),
                       kernel_initializer='glorot_normal', 
                       padding='same', 
                       activation='relu')

conv_2 = Convolution2D(conv_depth_1, 
                       (kernel_size, kernel_size), 
                       padding='same', 
                       activation='relu')

pool_1 = MaxPooling2D(pool_size = (pool_size, pool_size))

drop_1 = Dropout(drop_prob_1)

Arquitetura da Segunda Camada Convolucional

Conv_64 -> Conv_64 -> Pool_2x2 -> dropout_0.25

In [ ]:
conv_3 = Convolution2D(conv_depth_2, 
                       (kernel_size, kernel_size), 
                       padding='same', 
                       activation='relu')

conv_4 = Convolution2D(conv_depth_2, 
                       (kernel_size, kernel_size), 
                       padding='same', 
                       activation='relu')

pool_2 = MaxPooling2D(pool_size = (pool_size, pool_size))
drop_2 = Dropout(drop_prob_1)

Arquitetura da Terceira Camada Convolucional

Conv_128 -> Conv_128 -> Pool_2x2 -> dropout_0.25

In [ ]:
conv_5 = Convolution2D(conv_depth_3, 
                       (kernel_size, kernel_size), 
                       padding='same', 
                       activation='relu')

conv_6 = Convolution2D(conv_depth_3, 
                       (kernel_size, kernel_size), 
                       padding='same', 
                       activation='relu')

pool_3 = MaxPooling2D(pool_size = (pool_size, pool_size))

drop_3 = Dropout(drop_prob_1) 

Camadas Totalmente Conectadas

Flatten -> Hidden_256 -> dropout_1 -> Hidden_512 -> dropout_2 ->Dense [Output]

In [ ]:
flat = Flatten()
hidden = Dense(hidden_size1, activation='relu')
hidden1 = Dense(hidden_size2, activation='relu')
drop_3 = Dropout(drop_prob_2)
out1 = Dense(1)
out2 = Dense(1)
out3 = Dense(1)
out4 = Dense(1)

In [ ]:
# Para definir um modelo, basta especificar suas camadas de entrada e saída
model = Model(inputs = inp, outputs = [out1,out2,out3,out4]) 
print (model.summary())

In [ ]:
# Compilando o modelo
model.compile(loss = 'mean_squared_error', optimizer = 'adam')

In [ ]:
# Early-stopping
early_stop =  EarlyStopping(monitor = 'loss', min_delta = 0.001, patience = 5, verbose = 1, mode = 'min')
callbacks = [early_stop]

In [ ]:
# Fit do modelo
model.fit(X_train,
          [lx_train,ly_train,rx_train,ry_train],
          batch_size = batch_size, 
          epochs = num_epochs, 
          callbacks = callbacks,
          verbose = 1, 
          validation_split = 0.2)

In [ ]:
# Previsões

for i in range (0,10):
    lx_p=0
    ly_p=0
    rx_p=0
    ry_p=0
    index = randint(0,299)
    lx_p,ly_p,rx_p,ry_p = model.predict(X_test[index].reshape(1,1,180,180))
    lx_p=lx_p*(lx_max-lx_min)+lx_min
    ly_p=ly_p*(ly_max-ly_min)+ly_min
    rx_p=rx_p*(rx_max-rx_min)+rx_min
    ry_p=ry_p*(ry_max-ry_min)+ry_min
    
    display_img=X_test[index]*255.0
    
    fig,ax = plt.subplots(1)
    ax.imshow(display_img.reshape(180,180),cmap='gray') 

    rect1 = patches.Rectangle((lx_p,ly_p),2,2,linewidth=1,edgecolor='r',facecolor='none')
    rect2 = patches.Rectangle((rx_p,ry_p),2,2,linewidth=1,edgecolor='r',facecolor='none')

    ax.add_patch(rect1)
    ax.add_patch(rect2)

    plt.show()

O desempenho do modelo no conjunto de testes, não foi tão preciso, indicando que devemos ajustar o modelo, modificando os hiperparâmetros ou alterando a arquitetura do modelo. 

Salvando o modelo em fomato JSON e os pesos em formato HDF5

In [ ]:
model_json = model.to_json()
with open("modelo.json", "w") as json_file:
    json_file.write(model_json)

# Salvando o modelo no formato HDF5
model.save_weights("modelo.h5")
print("Modelo salvo com sucesso!")

# Fim